In [77]:
import json
import pandas as pd
from collections import defaultdict
import math

In [68]:
def read_data(path):
    rawdata=[]
    with open(path,"r",encoding="utf8") as infile:
        while True:
            linestr=infile.readline()
            if not linestr:
                break
            line=json.loads(linestr)
            rawdata.append(line)
    infile.close()
    return rawdata
        

In [69]:
User_Path="dataset\Peoria_Data\Peoria_User.json"
Review_Path="dataset\Peoria_Data\Peoria_Review.json"
Business_Path="dataset\Peoria_Data\Peoria_business.json"
Business_raw_data=read_data(Business_Path)
User_raw_data=read_data(User_Path)
Review_raw_data=read_data(Review_Path)


In [70]:
# Pre-processing the business data
Business_ids=[]
Business_names=[]
for shop in Business_raw_data:
    Business_ids.append(shop["business_id"])
    Business_names.append(shop["name"])
Business_df=pd.DataFrame()
Business_df["business_id"]=Business_ids
Business_df["name"]=Business_names



#Pre-processing the User data
User_ids=[]
User_names=[]
User_Review_Counts=[]
for user in User_raw_data:
    User_ids.append(user["name"])
    User_names.append(user["name"])
    User_ids.append(user["name"])

In [71]:
#Pre-processing the User data
User_ids=[]
User_names=[]
User_Review_Counts=[]
for user in User_raw_data:
    User_ids.append(user["user_id"])
    User_names.append(user["name"])
    User_Review_Counts.append(user["review_count"])
User_df=pd.DataFrame()
User_df["user_id"]=User_ids
User_df["name"]=User_names
User_df["Review_Counts"]=User_Review_Counts
User_df.head()

,user_id,name,Review_Counts
0,eSlOI3GhroEtcbaD_nFXJQ,Jason,2518
1,U4INQZOPSUaj8hMjLlZ3KA,Michael,4308
2,xZAmw5gihOVO4duMN2Ju6Q,Flynn,70
3,F78tJHr0qW6FsYBtWMmLpA,Christopher,267
4,TUE3NJYN4i7xyGVSIXMLvw,Kymberly,13


In [72]:
#Pre-processing the Review Data
Review_ids=[]
Review_User_ids=[]
Review_Business_ids=[]
Review_Stars=[]
for review in Review_raw_data:
    Review_ids.append(review["review_id"])
    Review_User_ids.append(review["user_id"])
    Review_Business_ids.append(review["business_id"])
    Review_Stars.append(review["stars"])
Review_df=pd.DataFrame()
Review_df["id"]=Review_ids
Review_df["user_id"]=Review_User_ids
Review_df["business_id"]=Review_Business_ids
Review_df["stars"]=Review_Stars
Review_df.head()

,id,user_id,business_id,stars
0,vxUvLAojLKEOftyJJweNEA,z4qE-IdYzvpzE8jP9oZYww,XvyXWS78g2soxHElyfgELw,5.0
1,65GF_UsW70rdxVde2ip6oA,GLsknAZX1t5lybLxuf9B3g,E6ADmD8pnHrfCqddyiaKXA,5.0
2,Y-KtijRX7iwjJWK9B48Ggg,qZB8BZ2ZzMF0VfIGhttu7w,D5aSQhj3uyX4Fj1B31EYRQ,1.0
3,EqwkO9ASFMVla1FJggTzCQ,Nx6BcJzyJdp4hP180O3xhg,1lxBO6qIHLcFl1WTQp-Ftg,5.0
4,ypkzI6POzwhj4RVxIo1Wxg,QRx4SICG18mRU0yocWsMMQ,1f7AEcZfKF7F4hvehrUqaw,2.0


In [73]:
total_num_of_user=len(Review_df.user_id.unique())
total_num_of_business=len(Review_df.business_id.unique())
print("There are total {} users and {} business in review set".format(total_num_of_user,total_num_of_business))

There are total 31757 users and 1919 business in review set


In [85]:
#Create item-user matrix in dictionary
item_user_dict = {}
dataDic_len = 0
for row in Review_df.itertuples():
    user, business, stars = getattr(row, 'user_id'), getattr(row, 'business_id'), getattr(row, 'stars')
    item_user_dict.setdefault(business, defaultdict(int))
    item_user_dict[business][user] = stars

In [83]:
# print(item_user_dict)
print(item_user_dict["XvyXWS78g2soxHElyfgELw"]["qZB8BZ2ZzMF0VfIGhttu7w"])

0


In [94]:
from scipy.linalg import norm
def simple_cosine_sim(a, b):
    if len(b) < len(a):
        a, b = b, a
    res = 0
    for key, a_value in a.items():
        res += a_value * b.get(key, 0)
    if res == 0:
        return 0
    try:
        res = res / norm(list(a.values())) / norm(list(b.values()))
    except ZeroDivisionError:
        res = 0
    return res 

In [ ]:
print(item_user_dict["XvyXWS78g2soxHElyfgELw"].values())

In [95]:
business_sim_matrix={}
for b1 in item_user_dict.keys():
    business_sim_matrix.setdefault(b1,defaultdict(int))
    for b2 in item_user_dict.keys():
        if b1==b2:
            continue
        cos_sim=simple_cosine_sim(item_user_dict[b1],item_user_dict[b2])
        if cos_sim==0:
            continue
        business_sim_matrix[b1][b2]=cos_sim

KeyboardInterrupt: 

In [86]:
#Record the time that the business has been reviewed, as popularity
business_popularity=defaultdict(int)
for key ,value in item_user_dict.items():
    business_popularity[key]=len(item_user_dict[key])

# Build business similarity matrix
business_sim_matrix={}
for b1 in item_user_dict.keys():
    business_sim_matrix.setdefault(b1,defaultdict(int))
    for b2 in item_user_dict.keys():
        if b1==b2:
            continue
        Cos_Numerator=0
        Cos_denominator=0
        for b1_user,b1_stars in item_user_dict[b1].items():
            Cos_Numerator+=b1_stars*item_user_dict[b2][b1_user]
        if(Cos_denominator!=0):
            b1_len=0
            b2_len=0
            for b1_stars in item_user_dict[b1].values():
                b1_len+=b1_stars**2
            for b2_stars in item_user_dict[b2].values():
                b2_len+=b2_stars**2
            Cos_denominator=math.sqrt(b1_len)*math.sqrt(b2_len)
            business_sim_matrix[b1][b2]=Cos_Numerator/Cos_denominator
        else:
            continue

KeyboardInterrupt: 